In [ ]:
#ferramentas para o problema
import pandas as pd
import numpy as np
import random
import ast

#ferramentas para o mapa interativo
import folium
from folium.plugins import AntPath

In [ ]:
def haversine(c1, c2):

    # Converte graus decimais para radians:
    lat1 = np.radians(c1[0])
    lon1 = np.radians(c1[1])
    lat2 = np.radians(c2[0])
    lon2 = np.radians(c2[1])
    
    # Implementando a formula haversine:
    dlon = np.subtract(lon2, lon1)
    dlat = np.subtract(lat2, lat1)

    a = np.add(np.power(np.sin(np.divide(dlat, 2)), 2),
               np.multiply(np.cos(lat1),
                           np.multiply(np.cos(lat2),
                                       np.power(np.sin(np.divide(dlon, 2)), 2))))

    c = np.multiply(2, np.arcsin(np.sqrt(a)))
    r = 6371

    return c*r

In [ ]:
def cost(permutation, cities):
    distance = 0
    for i, c1 in enumerate(permutation):
        if i == len(permutation) - 1:
            c2 = permutation[0]
        else:
            c2 = permutation[i + 1]
        distance += haversine(cities[c1], cities[c2])
    return distance

In [ ]:
def random_permutation(cities):
    perm = list(range(len(cities)))
    for i in range(len(perm)):
        r = random.randint(i, len(perm)-1)
        perm[r], perm[i] = perm[i], perm[r]
    return perm

In [ ]:
def initialise_pheromone_matrix(num_cities, naive_score):
    v = float(num_cities) / naive_score
    return [[v] * num_cities for _ in range(num_cities)]

In [ ]:
def calculate_choices(cities, last_city, exclude, pheromone, c_heur, c_hist):
    choices = []
    for i, coord in enumerate(cities):
        if i in exclude:
            continue
        prob = {'city': i}
        prob['history'] = pheromone[last_city][i] ** c_hist
        prob['distance'] = haversine(cities[last_city], coord)
        prob['heuristic'] = (1.0/prob['distance']) ** c_heur
        prob['prob'] = prob['history'] * prob['heuristic']
        choices.append(prob)
    return choices

In [ ]:
def select_next_city(choices):
    total_prob = sum(choice['prob'] for choice in choices)
    if total_prob == 0.0:
        return choices[random.randint(0, len(choices)-1)]['city']
    v = random.random()
    for choice in choices:
        v -= choice['prob'] / total_prob
        if v <= 0.0:
            return choice['city']
    return choices[-1]['city']

In [ ]:
def stepwise_const(cities, phero, c_heur, c_hist):
    perm = []
    perm.append(random.randint(0, len(cities) - 1))
    while len(perm) < len(cities):
        choices = calculate_choices(cities, perm[-1], perm, phero, c_heur, c_hist)
        next_city = select_next_city(choices)
        perm.append(next_city)
    return perm

In [ ]:
def decay_pheromone(pheromone, decay_factor):
    for array in pheromone:
        for i in range(len(array)):
            array[i] = (1.0 - decay_factor) * array[i]


In [ ]:
def update_pheromone(pheromone, solutions):
    for other in solutions:
        for i, x in enumerate(other['vector']):
            if i == len(other['vector']) - 1:
                y = other['vector'][0]
            else:
                y = other['vector'][i+1]
            pheromone[x][y] += (1.0 / other['cost'])
            pheromone[y][x] += (1.0 / other['cost'])

In [ ]:
def status_update(list, cities, iter):
    print(" > Iteração {}, Melhor={}".format(iter, list['cost']))
    y = [cities[i][0] for i in list['vector']]
    x = [cities[i][1] for i in list['vector']]

In [ ]:
def search(cities, max_it, num_ants, decay_factor, c_heur, c_hist):
    best = {'vector': random_permutation(cities)}
    best['cost'] = cost(best['vector'], cities)
    pheromone = initialise_pheromone_matrix(len(cities), best['cost'])
    for iter in range(max_it):
        solutions = []
        for _ in range(num_ants):
            candidate = {}
            candidate['vector'] = stepwise_const(cities, pheromone, c_heur, c_hist)
            candidate['cost'] = cost(candidate['vector'], cities)
            if candidate['cost'] < best['cost']:
                best = candidate
            solutions.append(candidate)
        decay_pheromone(pheromone, decay_factor)
        update_pheromone(pheromone, solutions)
        status_update(best, cities, iter+1)
    return best

In [ ]:
def mapa(list, cities):
    best_path = [cities[i] for i in list]
    mapObj = folium.Map(
            location=cities[0],
            zoom_start=13
    )
    for i, city in enumerate(cities):
        folium.Marker(city, popup=i).add_to(mapObj)
    AntPath(best_path).add_to(mapObj)
    mapObj.save('output/mapa.html')

In [ ]:
enderecos = pd.read_csv('REPO/enderecos_proc.csv')
enderecos['Coordenadas'] = enderecos['Coordenadas'].apply(ast.literal_eval)
df = enderecos.sample(30)
coordenadas = list(df['Coordenadas'])

### EXECUTANDO O PROBLEMA

In [ ]:
if __name__ == "__main__":
    # configuracao do problema

    max_it = 100 # numero de iteracoes
    num_ants = 30 # numero de formigas
    decay_factor = 0.9 # fator de decrescimento
    c_heur = 4 # coeficiente da heuristica
    c_hist = 1.0 # coeficiente do historico
    
    # executando o algoritmo
    best = search(coordenadas, max_it, num_ants, decay_factor, c_heur, c_hist)
    print(f"Completo. Melhor solução: c={best['cost']}, v={best['vector']}")

In [ ]:
vector = best['vector']
vector.append(best['vector'][0])


In [ ]:
output = pd.DataFrame(vector)

In [ ]:
mapa(vector, coordenadas)
output.to_csv('output/caminho.csv', index=False, header=False)